In [ ]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, WKTLayer

## Extract all the top public senior high schools listed on the bettereducation.com.au site

In [ ]:
best_senior_schools_df = pd.read_html(
    "https://bettereducation.com.au/school/secondary/qld/brisbane_top_government_secondary_schools.aspx"
)[0]

In [ ]:
best_senior_schools_clean_df = (
    best_senior_schools_df.dropna(axis=1)
    .drop(columns="Trend / Compare")
    .convert_dtypes()
)
best_senior_schools_clean_df["Better Education Percentile"] = (
    best_senior_schools_clean_df["Better Education Percentile"]
    .str.strip("%")
    .astype("int64")
)

# Adjust school names to match the abbreviations used in http://opendata.dete.qld.gov.au/
best_senior_schools_clean_df["School"] = (
    best_senior_schools_clean_df["School"]
    .str.replace("State High School", "SHS")
    .str.replace("High School", "High")
)

## Extract all the top public primary schools listed on the bettereducation.com.au site

In [ ]:
best_primary_schools_df = pd.read_html(
    "https://bettereducation.com.au/school/Primary/qld/brisbane_top_government_primary_schools.aspx"
)[0]

In [ ]:
best_primary_schools_clean_df = (
    best_primary_schools_df.dropna(axis=1)
    .drop(columns="Trend / Compare")
    .convert_dtypes()
)
best_primary_schools_clean_df["Better Education Percentile"] = (
    best_primary_schools_clean_df["Better Education Percentile"]
    .str.strip("%")
    .astype("int64")
)

# Adjust school names to match the abbreviations used in http://opendata.dete.qld.gov.au/
best_primary_schools_clean_df["School"] = (
    best_primary_schools_clean_df["School"]
    .str.replace("State School", "SS")
    .str.replace("State Infants and Primary School", "SIPS")
)

## Take the primary and senior high school catchment zome geometries from opendata.dete.qld.gov.au

In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
senior_high_schools_gdf = gpd.read_file(
    "http://opendata.dete.qld.gov.au/geo-data/2020/Senior-Secondary-Catchments-2020.kml",
    driver="KML",
)

In [ ]:
primary_schools_gdf = gpd.read_file(
    "http://opendata.dete.qld.gov.au/geo-data/2020/Primary-Catchments-2020.kml",
    driver="KML",
)

## Inner join the school catchment Geopandas dataframes with their corresponding best public school dataframes and plot resulting Geopandas dataframes

In [ ]:
best_senior_high_schools_gdf = senior_high_schools_gdf.merge(
    best_senior_schools_clean_df, how="inner", left_on="Name", right_on="School"
).drop(columns="School")

In [ ]:
best_senior_high_schools_gdf.plot()

In [ ]:
best_primary_schools_gdf = primary_schools_gdf.merge(
    best_primary_schools_clean_df, how="inner", left_on="Name", right_on="School"
).drop(columns="School")

In [ ]:
best_primary_schools_gdf.plot()

## Dissolve the best primary, and best senior high school geometries together and take the intersection of both to find areas that have good primary and senior high schools

In [ ]:
primary_schools_geometry = best_primary_schools_gdf[["geometry"]].dissolve().iloc[0,0]

In [ ]:
primary_schools_geometry

In [ ]:
senior_high_schools_geometry = best_senior_high_schools_gdf[["geometry"]].dissolve().iloc[0,0]

In [ ]:
senior_high_schools_geometry

In [ ]:
primary_and_senior_high_school_geometry = primary_schools_geometry.intersection(senior_high_schools_geometry)

In [ ]:
primary_and_senior_high_school_geometry

## Plot the combined school polygon over a leaflet map

In [ ]:
m = Map(center=(-27.5, 153), zoom=10)

In [ ]:
wlayer = WKTLayer(
    wkt_string=primary_and_senior_high_school_geometry.wkt,
)

In [ ]:
m.add_layer(wlayer)

In [ ]:
m